In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when
from pyspark import SparkConf, SparkContext

In [3]:
conf = SparkConf().setAppName("myApp")
sc = SparkContext(conf=conf)

spark = SparkSession(sc)
spark.conf.set("google.cloud.auth.service.account.enable", "true")
spark.conf.set("google.cloud.auth.service.account.json.keyfile", "config/sturdy-chimera-374511-73f24a51a3e4.json")

23/01/16 16:45:12 WARN Utils: Your hostname, conor-ubuntu-pc resolves to a loopback address: 127.0.1.1; using 192.168.0.29 instead (on interface wlp4s0)
23/01/16 16:45:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/16 16:45:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
df1 = spark.read.json("gs://project-pin-api/*.json")
df1.show()

+--------------+--------------------+----------+---------------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|      category|         description|downloaded| follower_count|           image_src|index|   is_image_or_video|       save_location|            tag_list|               title|           unique_id|
+--------------+--------------------+----------+---------------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|       finance|Description Coupo...|         1|              0|https://i.pinimg....| 5730|               image|Local save in /da...|Grocery Items,Gro...|Island Oasis Coup...|1e1f0c8b-9fcf-460...|
|diy-and-crafts|Keep the kids bus...|         1|           124k|https://i.pinimg....| 2863|               image|Local save in /da...|Summer Crafts For...|25 Super Fun Summ...|9bf39437-42a6-4f0...|
|  mens-fashion

In [36]:
df2 = spark.read.json("gs://project-pin-api/ea6864d7-1228-4059-9e2d-aa560197f0cf.json")
df2.show()

+--------------+--------------------+----------+--------------+--------------------+-----+-----------------+--------------------+--------------------+--------------------+--------------------+
|      category|         description|downloaded|follower_count|           image_src|index|is_image_or_video|       save_location|            tag_list|               title|           unique_id|
+--------------+--------------------+----------+--------------+--------------------+-----+-----------------+--------------------+--------------------+--------------------+--------------------+
|diy-and-crafts|Keep the kids bus...|         1|          124k|https://i.pinimg....| 2863|            image|Local save in /da...|Summer Crafts For...|25 Super Fun Summ...|9bf39437-42a6-4f0...|
+--------------+--------------------+----------+--------------+--------------------+-----+-----------------+--------------------+--------------------+--------------------+--------------------+



In [34]:
# Create a dictionary that maps columns to bad values
bad_values = {
    "title": "No Title Data Available",
    "description": "No description available Story format",
    "follower_count": "User Info Error",
    "tag_list": "N,o, ,T,a,g,s, ,A,v,a,i,l,a,b,l,e",
    "image_src": "Image src error."
}

# Iterate over the dictionary and replace the bad values with null
for column, bad_value in bad_values.items():
    df1 = df1.withColumn(column, when(col(column) == bad_value, None).otherwise(col(column)))

In [35]:
df1.show()

+------------+-----------+----------+--------------+---------+-----+--------------------+--------------------+--------+-----+--------------------+
|    category|description|downloaded|follower_count|image_src|index|   is_image_or_video|       save_location|tag_list|title|           unique_id|
+------------+-----------+----------+--------------+---------+-----+--------------------+--------------------+--------+-----+--------------------+
|mens-fashion|       null|         0|          null|     null| 7528|multi-video(story...|Local save in /da...|    null| null|fbe53c66-3442-477...|
+------------+-----------+----------+--------------+---------+-----+--------------------+--------------------+--------+-----+--------------------+



In [17]:
spark.stop()
sc.stop()